In [1]:
import numpy  as np
import pandas as pd

import matplotlib.pyplot as plt

import remap_values      as rv
import df_visualizations as dv

In [6]:
# price_doc new sale price

In [2]:
train_df = pd.read_csv( 'inp_data/train.csv' )
test_df  = pd.read_csv( 'inp_data/test.csv'  )

target_price = np.log10( train_df       ['price_doc'] .copy() )
train_df     =           train_df.drop( ['price_doc'] , axis=1 )

big_df = train_df.append( test_df ).copy()

In [3]:
big_df['N_build']    = big_df[ ['build_count_wood','build_count_frame','build_count_mix',
                                'build_count_slag','build_count_foam','build_count_block',
                                'build_count_panel','build_count_monolith','build_count_brick'] ].sum( axis=1 )

big_df['Good_build'] = (big_df['build_count_wood'] + big_df['build_count_frame'] + big_df['build_count_mix'] + 
                        big_df['build_count_slag'] + big_df['build_count_foam']) / big_df['N_build']

big_df['Bad_build' ] = (big_df['build_count_block']+ big_df['build_count_panel'] + big_df['build_count_monolith'] + 
                        big_df['build_count_brick'] ) / big_df['N_build']

big_df['Good_build'] = big_df['Good_build'].fillna( big_df['Good_build'].median() )
big_df['Bad_build' ] = big_df['Bad_build' ].fillna( big_df['Bad_build' ].median() )


# Do the above replace for these, make list of those prob
power_plants = ['thermal_power_plant_raion','nuclear_reactor_raion']
ruin_list    = ['incineration_raion','oil_chemistry_raion','radiation_raion']
misc_list    = ['culture_objects_top_25','big_market_raion','detention_facility_raion','big_road1_1line']

big_list = power_plants+ruin_list+misc_list

for item in big_list:
    big_df[item] = big_df[item].replace( {'no':0,'yes':1} ).fillna(0)
    
big_df['product_type'] = big_df['product_type'].replace( {'Investment':0, 'OwnerOccupier':1} ).fillna(0)
big_df['state'       ] = big_df['state'       ].replace( {33.:3.}).fillna(0.)
big_df['material'    ] = big_df['material'    ].replace( {3:0})
big_df['material'    ] = big_df['material'    ].replace( {6:3}).fillna(0)


big_df['floor'    ] = big_df['floor'    ].fillna( 0 )
big_df['state'    ] = big_df['state'    ].fillna( 0 )
big_df['max_floor'] = big_df['max_floor'].fillna( 0 )


adj = 'prom_part_5000'
big_df[adj] = big_df[adj].fillna( big_df[adj].median() )

adj = 'life_sq'
big_df[adj] = big_df[adj].fillna( big_df[adj].median() )

adj = 'num_room'
big_df[adj] = big_df[adj].fillna( big_df[adj].median() )

adj = 'kitch_sq'
big_df[adj] = big_df[adj].fillna( big_df[adj].median() )

big_df[]2.00520090e+07


#'timestamp','sub_area','ecology'

In [4]:
# This stuff had near 0 correlation to final price
drop_list_age      = ['0_6_all','0_6_male','0_6_female','7_14_all','7_14_male','7_14_female','0_17_all','0_17_male','0_17_female','16_29_all','16_29_male','16_29_female','0_13_all','0_13_male','0_13_female']
drop_list_aGroup   = ['young_all', 'young_male', 'young_female', 'work_all', 'work_male', 'work_female', 'ekder_all', 'ekder_male', 'ekder_female']
drop_list_pop      = ['full_all','male_f','female_f']
drop_list_neigh    = ['raion_popul']
drop_list_green    = ['green_part_3000','green_part_2000','green_part_1500','green_part_1000','green_part_500']
drop_list_school   = ['preschool_km','school_education_centers_top_20_raion','school_quota','preschool_quota','school_quota','children_preschool','preschool_education_centers_raion']
drop_list_build    = ['build_count_wood','build_count_frame','build_count_mix','build_count_slag','build_count_foam','build_count_block','build_count_panel','build_count_monolith','build_count_brick','build_count_before_1920','build_count_1921-1945','build_count_1946-1970','build_count_1971-1995','build_count_after_1995']
drop_list_auto     = ['metro_km_avto','metro_min_walk','metro_km_walk','ID_metro']
drop_list_railroad = ['railroad_station_avto_km', 'railroad_station_walk_min', 'railroad_station_walk_km', 'ID_railroad_station_avto', 'ID_railroad_station_walk', 'railroad_1line','railroad_terminal_raion','ID_railroad_terminal'] 
drop_list_metro    = ['ID_metro', 'metro_km_avto', 'metro_min_walk', 'metro_km_walk']
drop_list_sport    = ['sport_objects_raion', 'public_transport_station_min_walk', 'sport_count_500', 'sport_count_1000', 'sport_count_1500', 'sport_count_2000', 'sport_count_3000']
drop_list_office   = ['office_raion',  'office_count_500', 'office_count_5000', 'office_sqm_500', 'office_count_1000', 'office_sqm_1000', 'office_count_1500', 'office_sqm_1500', 'office_count_2000', 'office_sqm_2000', 'office_count_3000', 'office_sqm_3000' ]


drop_list_misc   = ['hospital_beds_raion','university_top_20_raion','public_transport_station_min_walk','additional_education_raion',
                    'culture_objects_top_25_raion','shopping_centers_raion','water_1line','ID_bus_terminal']
# Undropped
drop_list_power  = ['thermal_power_plant_raion']
drop_list_ruin   = ['incineration_raion','oil_chemistry_raion','radiation_raion']
drop_list_market = ['market_count_500','market_count_1000','market_count_1500','market_count_2000','market_count_3000']
drop_list_info   = ['raion_build_count_with_material_info','raion_build_count_with_builddate_info']
drop_list_road   = ['ID_big_road1', 'ID_big_road2']
drop_list_indus  = ['prom_part_500', 'prom_part_1000', 'prom_part_1500', 'prom_part_2000', 'prom_part_3000']
drop_list_mall   = ['trc_count_500', 'trc_sqm_500', 'trc_count_1000', 'trc_sqm_1000', 'trc_count_1500', 'trc_sqm_1500', 'trc_count_2000', 'trc_sqm_2000', 'trc_count_3000', 'trc_sqm_3000']

drop_list_church     = ['church_count_500', 'church_count_1000', 'church_count_1500', 'church_count_2000', 'church_count_3000']
drop_list_mosque     = ['mosque_count_500', 'mosque_count_1000', 'mosque_count_1500', 'mosque_count_2000', 'mosque_count_3000']
drop_list_leisure    = ['leisure_count_500', 'leisure_count_1000', 'leisure_count_1500', 'leisure_count_2000', 'leisure_count_3000']
drop_list_big_church = ['big_church_count_500', 'big_church_count_1000', 'big_church_count_1500', 'big_church_count_2000', 'big_church_count_3000']


super_list = drop_list_age    + drop_list_aGroup  + drop_list_pop        + drop_list_neigh    + drop_list_green  + \
             drop_list_school + drop_list_build   + drop_list_auto       + drop_list_railroad + drop_list_metro  + \
             drop_list_sport  + drop_list_office  + drop_list_power      + drop_list_ruin     + drop_list_market + \
             drop_list_info   + drop_list_road    + drop_list_indus      + drop_list_mall     + drop_list_church + \
             drop_list_mosque + drop_list_leisure + drop_list_big_church + \
             drop_list_misc

big_df = big_df.drop( super_list, axis=1 )

# First to do:
foo = big_df['cafe_count_5000'           ].copy()
bar = big_df['cafe_count_5000_price_high'].copy()
# Drop everyhing else
big_df = big_df.drop( big_df.columns[ big_df.columns.str.contains('cafe') ], axis=1 )
big_df['cafe_count_5000']            = foo.copy()
big_df['cafe_count_5000_price_high'] = bar.copy()

In [6]:
print big_df.shape
for col in big_df.columns.values:
    print col

(38133, 91)
id
timestamp
full_sq
life_sq
floor
max_floor
material
build_year
num_room
kitch_sq
state
product_type
sub_area
area_m
green_zone_part
indust_part
children_school
school_education_centers_raion
healthcare_centers_raion
culture_objects_top_25
big_market_raion
nuclear_reactor_raion
detention_facility_raion
metro_min_avto
kindergarten_km
school_km
park_km
green_zone_km
industrial_km
water_treatment_km
cemetery_km
incineration_km
railroad_station_avto_min
public_transport_station_km
water_km
mkad_km
ttk_km
sadovoe_km
bulvar_ring_km
kremlin_km
big_road1_km
big_road1_1line
big_road2_km
railroad_km
zd_vokzaly_avto_km
bus_terminal_avto_km
oil_chemistry_km
nuclear_reactor_km
radiation_km
power_transmission_line_km
thermal_power_plant_km
ts_km
big_market_km
market_shop_km
fitness_km
swim_pool_km
ice_rink_km
stadium_km
basketball_km
hospice_morgue_km
detention_facility_km
public_healthcare_km
university_km
workplaces_km
shopping_centers_km
office_km
additional_education_km
big_church_k

In [12]:
print big_df['product_type'].unique()

big_df['product_type'].isnull().sum()
#dv.hist_plot( big_df, 'product_type' )
#dv.avg_plot( big_df, 'product_type' )

['Investment' 'OwnerOccupier' nan]


33

In [39]:
dv.hist_plot( foo, 'state' )

In [19]:
print foo['state'].unique()

[ nan   3.   1.   2.   4.  33.]


In [5]:
null_cols = big_df.ix[:,big_df.isnull().sum() > 0 ]

In [26]:
print null_cols.shape
print null_cols.isnull().sum()

new_cols     = null_cols.copy()
new_cols['build_year'] = new_cols['build_year'].replace( {2.00520090e+07:2005} )

new_cols[ new_cols['build_year'] < 1000 ] = new_cols['build_year'].mean()
new_cols[ new_cols['build_year'] > 2020 ] = new_cols['build_year'].mean()



print new_cols.isnull().sum()

(38133, 1)
build_year    14654
dtype: int64
build_year    14654
dtype: int64


In [27]:
new_cols[  new_cols['build_year']<3000 ]['build_year'].unique()

array([ 1907.        ,  1980.        ,  2014.        ,  1970.        ,
        1982.        ,  2013.        ,  2004.        ,  2003.        ,
        1957.        ,  1986.        ,  1960.        ,  1995.        ,
        1979.        ,  1975.        ,  1987.        ,  1962.        ,
        1969.        ,  1993.        ,  1996.        ,  1972.        ,
        2011.        ,  1965.        ,  2010.        ,  1985.        ,
        2006.        ,  1961.        ,  1971.        ,  1978.        ,
        1966.        ,  1967.        ,  2000.        ,  1964.        ,
        1977.        ,  1983.        ,  1968.        ,  1974.        ,
        2008.        ,  1959.        ,  2007.        ,  1984.        ,
        1976.        ,  1997.        ,  1989.        ,  1958.        ,
        1988.        ,  2012.        ,  1990.        ,  1946.        ,
        1917.        ,  2002.        ,  2005.        ,  2001.        ,
        1963.        ,  1954.        ,  1951.        ,  1981.        ,
      

In [33]:
new_cols['build_year'].hist( bins=50 )
plt.show()
#dv.hist_plot( null_cols, 'build_year' )
#dv.plot_avg( null_cols, 'material', 'life_sq')

In [18]:
foo = big_df.drop(['timestamp','sub_area','ecology'],axis=1).copy()
foo['product_type'] = big_df['product_type'].replace( {'Investment':0, 'OwnerOccupier':1} ).fillna(-1)

print foo.corrwith( foo['product_type'] ).sort_values(ascending=False)

product_type                      1.000000
workplaces_km                     0.468294
detention_facility_km             0.463551
university_km                     0.455659
zd_vokzaly_avto_km                0.447242
market_shop_km                    0.430649
museum_km                         0.425267
area_m                            0.421861
basketball_km                     0.420804
office_km                         0.418978
nuclear_reactor_km                0.418613
ttk_km                            0.415553
fitness_km                        0.406343
sadovoe_km                        0.401508
bulvar_ring_km                    0.393121
stadium_km                        0.389909
catering_km                       0.382903
kremlin_km                        0.380495
swim_pool_km                      0.379068
theater_km                        0.373133
oil_chemistry_km                  0.371542
thermal_power_plant_km            0.371112
public_healthcare_km              0.359304
hospice_mor

In [8]:
print big_df.drop('timestamp',axis=1).corrwith( target_price )

TypeError: ("unsupported operand type(s) for /: 'str' and 'int'", u'occurred at index product_type')

In [57]:
big_df['ecology'].unique()

array(['good', 'excellent', 'poor', 'satisfactory', 'no data'], dtype=object)

In [58]:
dv.hist_plot( big_df, 'ecology' )

In [63]:
foo = big_df[['ecology','theater_km']].copy()
foo['Price'] = target_price.copy()
dv.plot_avg( foo, 'ecology', 'Price' )

In [69]:
foo = big_df.ix[:, big_df.columns.str.contains('count') ].copy()#/big_df.ix[].copy()
#foo =    foo.ix[:,    foo.columns.str.contains('count')].copy()
#foo = big_df[ ['raion_popul','green_zone_part','indust_part'] ].copy()
#print foo.head()
print foo.columns.values


foo['Price']      = target_price.copy()


#foo['ecology'] = foo['ecology'].replace( {'no data':0,'poor':1,'satisfactory':2,'good':3,'excellent':4} ).fillna(0)


print foo.corrwith( target_price ).sort_values()
#print foo.corr()


#dv.corr_plot( foo, focus='Price' )
#dv.corr_plot( foo[['office_km', 'office_count_5000', 'office_sqm_5000','Price']] ,focus='Price' )

['raion_build_count_with_material_info'
 'raion_build_count_with_builddate_info' 'trc_count_500'
 'big_church_count_500' 'church_count_500' 'mosque_count_500'
 'leisure_count_500' 'market_count_500' 'trc_count_1000'
 'big_church_count_1000' 'church_count_1000' 'mosque_count_1000'
 'leisure_count_1000' 'market_count_1000' 'trc_count_1500'
 'big_church_count_1500' 'church_count_1500' 'mosque_count_1500'
 'leisure_count_1500' 'market_count_1500' 'trc_count_2000'
 'big_church_count_2000' 'church_count_2000' 'mosque_count_2000'
 'leisure_count_2000' 'market_count_2000' 'trc_count_3000'
 'big_church_count_3000' 'church_count_3000' 'mosque_count_3000'
 'leisure_count_3000' 'market_count_3000' 'trc_count_5000'
 'big_church_count_5000' 'church_count_5000' 'mosque_count_5000'
 'leisure_count_5000' 'sport_count_5000' 'market_count_5000'
 'cafe_count_5000' 'cafe_count_5000_price_high']
church_count_500                         0.008802
mosque_count_500                         0.013499
big_church_co

In [22]:
(big_df['0_6_male']/big_df['0_6_all']).hist( bins=20 )
plt.show()

In [15]:
'''
timestamp: date of transaction
full_sq: total area in square meters, including loggias, balconies and other non-residential areas
life_sq: living area in square meters, excluding loggias, balconies and other non-residential areas
floor: for apartments, floor of the building
max_floor: number of floors in the building
material: wall material
build_year: year built
num_room: number of living rooms
kitch_sq: kitchen area
state: apartment condition
product_type: owner-occupier purchase or investment
'''
home_cols = ['full_sq', 'life_sq', 'kitch_sq', 'num_room',
             'floor', 'max_floor', 
             'material', 'build_year', 'state', 
             'product_type']

In [16]:
'''
raion=sub_area
sub_area: name of the district
full_all: subarea population
male_f, female_f: subarea population by gender
young_*: population younger than working age
work_*: working-age population
ekder_*: retirement-age population
n_m_{all|male|female}: population between n and m years old
build_count_*: buildings in the subarea by construction type or year
x_count_500: the number of x within 500m of the property
x_part_500: the share of x within 500m of the property
_sqm_: square meters
cafe_count_d_price_p: number of cafes within d meters of the property that have an average bill under p RUB
trc_: shopping malls
prom_: industrial zones
green_: green zones
metro_: subway
_avto_: distances by car
mkad_: Moscow Circle Auto Road
ttk_: Third Transport Ring
sadovoe_: Garden Ring
bulvar_ring_: Boulevard Ring
kremlin_: City center
zd_vokzaly_: Train station
oil_chemistry_: Dirty industry
ts_: Power plant
'''

'\nsub_area: name of the district\nfull_all: subarea population\nmale_f, female_f: subarea population by gender\nyoung_*: population younger than working age\nwork_*: working-age population\nekder_*: retirement-age population\nn_m_{all|male|female}: population between n and m years old\nbuild_count_*: buildings in the subarea by construction type or year\nx_count_500: the number of x within 500m of the property\nx_part_500: the share of x within 500m of the property\n_sqm_: square meters\ncafe_count_d_price_p: number of cafes within d meters of the property that have an average bill under p RUB\ntrc_: shopping malls\nprom_: industrial zones\ngreen_: green zones\nmetro_: subway\n_avto_: distances by car\nmkad_: Moscow Circle Auto Road\nttk_: Third Transport Ring\nsadovoe_: Garden Ring\nbulvar_ring_: Boulevard Ring\nkremlin_: City center\nzd_vokzaly_: Train station\noil_chemistry_: Dirty industry\nts_: Power plant\n'